In [1]:
cd /home/ubuntu/FedEM/

/home/ubuntu/FedEM


In [2]:
# Import General Libraries
import os
import argparse
import torch
import copy
import pickle
import random
import numpy as np
import pandas as pd

# Import FedEM based Libraries
from utils.utils import *
from utils.constants import *
from utils.args import *
from run_experiment import *
from models import *

# Import Transfer Attack
from transfer_attacks.Personalized_NN import *
from transfer_attacks.Params import *
from transfer_attacks.Transferer import *
from transfer_attacks.Args import *
from transfer_attacks.TA_utils import *
from transfer_attacks.Boundary_Transferer import *

In [3]:
# Generating Empty Aggregator to be loaded 

setting = 'FedEM'

if setting == 'FedEM':
    nL = 3
else:
    nL = 1

# Manually set argument parameters
args_ = Args()
args_.experiment = "cifar10"
args_.method = setting
args_.decentralized = False
args_.sampling_rate = 1.0
args_.input_dimension = None
args_.output_dimension = None
args_.n_learners= nL
args_.n_rounds = 10
args_.bz = 128
args_.local_steps = 1
args_.lr_lambda = 0
args_.lr =0.03
args_.lr_scheduler = 'multi_step'
args_.log_freq = 10
args_.device = 'cuda'
args_.optimizer = 'sgd'
args_.mu = 0
args_.communication_probability = 0.1
args_.q = 1
args_.locally_tune_clients = False
args_.seed = 1234
args_.verbose = 1
args_.save_path = 'weights/cifar/dummy/'
args_.validation = False

# Generate the dummy values here
aggregator, clients = dummy_aggregator(args_, num_user=40)

==> Clients initialization..
===> Building data iterators..


100%|██████████████████████████████████████████| 80/80 [00:00<00:00, 198.01it/s]


===> Initializing clients..


100%|███████████████████████████████████████████| 80/80 [00:33<00:00,  2.41it/s]


==> Test Clients initialization..
===> Building data iterators..


0it [00:00, ?it/s]


===> Initializing clients..


0it [00:00, ?it/s]


++++++++++++++++++++++++++++++
Global..
Train Loss: 2.292 | Train Acc: 12.195% |Test Loss: 2.292 | Test Acc: 12.291% |
++++++++++++++++++++++++++++++++++++++++++++++++++
################################################################################


In [22]:
y = clients[0].train_iterator.dataset.targets
y2 = copy.deepcopy(y)
class_count = 10

In [23]:
y2 = class_count - y - 1

In [19]:
for i in range(y.shape[0]):
    y2[i] = class_count - y[i] - 1

In [24]:
y

tensor([5, 2, 6, 0, 1, 4, 6, 1, 2, 9, 6, 1, 0, 1, 1, 6, 6, 5, 5, 5, 1, 5, 0, 4,
        6, 9, 9, 2, 9, 5, 5, 1, 2, 5, 4, 2, 3, 6, 9, 6, 9, 6, 9, 6, 9, 1, 6, 2,
        9, 9, 1, 1, 2, 1, 1, 4, 9, 5, 9, 1, 5, 1, 6, 2, 9, 1, 5, 5, 9, 2, 6, 1,
        0, 0, 3, 4, 9, 1, 2, 4, 4, 4, 7, 6, 1, 1, 5, 4, 1, 2, 4, 4, 1, 6, 5, 4,
        0, 2, 0, 0, 4, 2, 6, 6, 9, 9, 6, 5, 0, 4, 1, 4, 5, 2, 6, 4, 0, 9, 1, 5,
        6, 0, 1, 6, 7, 8, 1, 4, 4, 4, 4, 6, 3, 6, 2, 9, 6, 0, 0, 5, 1, 0, 1, 9,
        9, 2, 2, 4, 5, 2, 2, 9, 1, 5, 0, 1, 9, 2, 1, 2, 6, 1, 3, 2, 4, 5, 9, 6,
        2, 0, 5, 4, 1, 2, 9, 0, 9, 0, 2, 5, 2, 9, 1, 4, 5, 0, 9, 9, 5, 0, 1, 5,
        0, 9, 5, 4, 2, 1, 6, 2, 4, 9, 6, 1, 1, 9, 1, 9, 5, 2, 6])

In [25]:
y2

tensor([4, 7, 3, 9, 8, 5, 3, 8, 7, 0, 3, 8, 9, 8, 8, 3, 3, 4, 4, 4, 8, 4, 9, 5,
        3, 0, 0, 7, 0, 4, 4, 8, 7, 4, 5, 7, 6, 3, 0, 3, 0, 3, 0, 3, 0, 8, 3, 7,
        0, 0, 8, 8, 7, 8, 8, 5, 0, 4, 0, 8, 4, 8, 3, 7, 0, 8, 4, 4, 0, 7, 3, 8,
        9, 9, 6, 5, 0, 8, 7, 5, 5, 5, 2, 3, 8, 8, 4, 5, 8, 7, 5, 5, 8, 3, 4, 5,
        9, 7, 9, 9, 5, 7, 3, 3, 0, 0, 3, 4, 9, 5, 8, 5, 4, 7, 3, 5, 9, 0, 8, 4,
        3, 9, 8, 3, 2, 1, 8, 5, 5, 5, 5, 3, 6, 3, 7, 0, 3, 9, 9, 4, 8, 9, 8, 0,
        0, 7, 7, 5, 4, 7, 7, 0, 8, 4, 9, 8, 0, 7, 8, 7, 3, 8, 6, 7, 5, 4, 0, 3,
        7, 9, 4, 5, 8, 7, 0, 9, 0, 9, 7, 4, 7, 0, 8, 5, 4, 9, 0, 0, 4, 9, 8, 4,
        9, 0, 4, 5, 7, 8, 3, 7, 5, 0, 3, 8, 8, 0, 8, 0, 4, 7, 3])